"Preciso de um script em Python que leia uma planilha de vendas e gere um relatório consolidado.
A planilha tem as colunas:

Data | Produto | Vendedor | Quantidade | Valor 

_Considere a coluna Valor como Valor Total da venda_

O relatório final deve conter:
- Faturamento total
- Vendas por produto (quantidade e valor)
- Ranking de vendedores (do maior para o menor em valor vendido)

Uma planilha Excel com o relatório dividido em abas"

Entrega esperada:
- Código em Python bem organizado
- Arquivo Excel de saída com pelo menos 3 abas:
- Resumo geral
- Resumo por produto
- Ranking de vendedores
- Um pequeno README explicando como rodar

In [1]:
import pandas as pd
import xlsxwriter as xlsx

In [2]:
#Leitura do excel
df = pd.read_excel("vendas.xlsx", engine= "openpyxl")
df.head()

,Data,Produto,Vendedor,Quantidade,Valor
0,2025-08-28,Colar Artesanal,João,5,355
1,2025-09-07,Difusor,Ana,2,170
2,2025-09-11,Colar Artesanal,Beatriz,4,128
3,2025-09-07,Suporte de Macramê,Ana,4,200
4,2025-09-01,Vela Aromática,Beatriz,1,80


In [3]:
#Validacoes iniciais
colunas_necessarias = ["Data", "Produto", "Vendedor", "Quantidade", "Valor"]
for col in colunas_necessarias:
    if col not in df.columns:
        print(f"Atenção! Coluna {col} não encontrada")

df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
df["Quantidade"] = pd.to_numeric(df["Quantidade"], errors="coerce")
df["Valor"] = pd.to_numeric(df["Valor"], errors="coerce")

print(df.dtypes)

Data          datetime64[ns]
Produto               object
Vendedor              object
Quantidade             int64
Valor                  int64
dtype: object


In [4]:
#KPIs de resumo
faturamento_total = df["Valor"].sum()
quantidade_total = df["Quantidade"].sum()
ticket_medio = faturamento_total/quantidade_total

produtos_vendidos = df["Produto"].nunique()
vendedores = df["Vendedor"].nunique()

In [5]:
#Resumo por produto
df_por_produto = df.groupby("Produto").agg({
    "Quantidade":"sum",
    "Valor":"sum"})
df_por_produto["PrecoMedio"] = df_por_produto["Valor"]/df_por_produto["Quantidade"]
df_por_produto.head()

,Quantidade,Valor,PrecoMedio
Produto,,,
Colar Artesanal,61,4098,67.180328
Copo de Cimento,44,3067,69.704545
Difusor,63,4916,78.031746
Suporte de Macramê,69,4561,66.101449
Vela Aromática,66,4812,72.909091


In [6]:
#Resumo por vendedor
df_por_vendedor = df.groupby("Vendedor").agg({
    "Quantidade":"sum",
    "Valor":"sum"})
df_por_vendedor = df_por_vendedor.sort_values("Valor", ascending=False)
df_por_vendedor["Ranking"] = range(1, len(df_por_vendedor) +1)
print(df_por_vendedor)

          Quantidade  Valor  Ranking
Vendedor                            
Carlos            80   5477        1
João              64   5124        2
Ana               60   3915        3
Beatriz           51   3655        4
Marcos            48   3283        5


In [7]:
#KPIs do resumo em um dataframe
resumo_geral = pd.DataFrame({
    "Métrica":["Faturamento Total", "Quantidade Total", "Ticket Médio", "Número de Produtos", "Número de Vendedores"],
    "Valor":[faturamento_total, quantidade_total, ticket_medio, produtos_vendidos, vendedores]
})

In [10]:
#Escrevendo o excel
with pd.ExcelWriter("relatorio_vendas.xlsx", engine="xlsxwriter") as writer:
    resumo_geral.to_excel(writer, sheet_name="Resumo_Geral", index=False)
    df_por_produto.to_excel(writer, sheet_name="Resumo_Produto", index=False)
    df_por_vendedor.to_excel(writer, sheet_name="Ranking_Vendedores", index=False)

    #Formatacao excel
    workbook = writer.book
    worksheet = writer.sheets["Resumo_Geral"]
    formato_moeda = workbook.add_format({'num_format':'R$ #,##0.00'})
    formato_inteiro = workbook.add_format({'num_format':'0'})
    
    for row, valor in enumerate(resumo_geral["Valor"]):
        if row in [0,2]:
            worksheet.write(row+1, 1, valor, formato_moeda)
        else:
            worksheet.write(row+1, 1, valor, formato_inteiro)
    
    worksheet = writer.sheets["Resumo_Produto"]
    worksheet.set_column('C:C', 15, formato_moeda)
    worksheet.set_column('D:D', 15, formato_moeda)
    
    worksheet = writer.sheets["Ranking_Vendedores"]
    worksheet.set_column('C:C', 15, formato_moeda)